<strong> <h1 style='text-align: center;'> UI CODE <br>FOR QUERYING CELLAR TRIPLESTORE ASSESSMENTS </h1> <strong/>

<strong> <h2>1. Import resources </h2> <strong/>
    
<p>The following block is importing:</p>
     <ul>
      <li>The module with the global variables to allow you connect to the API that performs the Cellar Triplestore query and also have a query collection.</li>
      <li>The libraries needed to create the UI components and format results.</li>
    </ul>

In [1]:
import sys
sys.path.append('../../')
import requests
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout
import pandas as pd
import json
from pprint import pprint
import cfg.ctt as ctt
from IPython.display import display, HTML
from itables import init_notebook_mode
init_notebook_mode(all_interactive = True)

<IPython.core.display.Javascript object>

<strong> <h2>2. Variables </h2> <strong/>
    
<p>The following block is defining the global variables necessary to this UI module</p>

In [2]:
TAB_IDX = 0

<strong> <h2>3. UI component </h2> <strong/>
    
<p>The following block makes use of <a href='https://ipywidgets.readthedocs.io/en/latest/index.html'>Ipywidgets Library</a> for the creation of the ui component used by the user to interact with SPARQL queries. The widgets used are:</p>
    <ul>
      <li>1. <u>Tab</u> it is used as container to hold other widgets called children. Allows to distribute the content by tabs
          depending on what to show the user for each tab.</li>
      <li>2. <u>VBox</u> it is used as Layout of conatiner widtget to organise the children widgets vertically</li>
      <li>2. <u>HTML</u> is used to display a string value. Allows format it with css styles. </li>
      <li>3. <u>Textarea</u> is a widget that accept the user's input. </li>
      <li>4. <u>Button</u> is used to trigger an event</li>
      <li>4. <u>Output</u> is the event that captures the return of the function that sends the query to the API to make SPARQL queries 
          to the CELLAR Triplestore Endpoint.</li>
    </ul>

In [3]:
list_title_tab = [widgets.HTML(value = dict_value['title'], layout=Layout(width='10%x')) 
                        for dict_value in ctt.PROCURER_USE_CASES]

list_description_tab = [widgets.HTML(value = dict_value['description'], layout=Layout(width='10%x')) 
                        for dict_value in ctt.PROCURER_USE_CASES]

list_text_area = [widgets.Textarea(value = dict_value['query_value'], placeholder='Type a query', layout=Layout(width='95%',height='200px'))
           for dict_value in ctt.PROCURER_USE_CASES]

button = widgets.Button(description="Run")

tab_content = [widgets.VBox([list_title_tab[i], list_description_tab[i], list_text_area[i], button]) 
               for i, tab_widget in enumerate(list_text_area)]

tab= widgets.Tab()
tab.children = tab_content
[tab.set_title(i, dict_value['name']) for i, dict_value in enumerate(ctt.PROCURER_USE_CASES)]

[None, None]

<strong> <h2>4. Functions </h2> <strong/>
    
<p>The following code blocks define the functions needed to assign the values to the tab widget and communicate with the Assessments API Microservice developed by CAMSS Team:</p>

<strong> <h4> a. get_tab_index function:</h4> <strong/>
<p> Method to get the correct Output widget based on the index of the chosen tab. </p

In [4]:
def get_tab_index(widget):
    global TAB_IDX
    TAB_IDX = widget['new']
    pprint(TAB_IDX)

<strong> <h4> b. send_query function:</h4> <strong/>
<p> Method to run the Assessments API Microservice that sends SPARQL queries to the <a href='https://op.europa.eu/en/publication-detail/-/publication/50ecce27-857e-11e8-ac6a-01aa75ed71a1/language-en![image.png](attachment:5b22dba6-96ba-4400-b673-bbe75d4ffc94.png)![image.png](attachment:e7844137-cf8f-4f96-8352-da68895f17b9.png)![image.png](attachment:8b52e53f-2939-4729-b535-98b4f70f2874.png)'>CELLAR's Virtuoso endpoint.</a></p

In [5]:
output= widgets.Output()

@output.capture()
def send_query(q):
    global TAB_IDX
    query = list_text_area[TAB_IDX].value
    params = {'sparql_query': query}
    msg = requests.post(url= ctt.URL_MICROSERVICE, params= params)
    status_code = msg.status_code
    msg_content = msg.json()
    if status_code >= 200 and status_code < 299:
        df_query_response = pd.DataFrame(msg_content["list_result"])
        display(HTML(df_query_response.to_html()))
    else:
        pprint(msg_content)

<strong> <h4> c. format_results_query function:</h4> <strong/>
<p> Method to format query results into interactive tables to allow the user to interact with their content</p

<strong> <h2>5. Display UI </h2> <strong/>

In [6]:
display(tab, output)

tab.observe(get_tab_index, names='selected_index')
button.on_click(send_query)

Output()